In [24]:
import pandas as pd
import call_api_service as ca
import twitch_api_login
import authenticator

## Open the file of streamers

In [25]:
top_2500 = pd.read_csv('../top_2500_streamer.csv', header=None)
top_2500[0]

0         aminematue
1         RebeuDeter
2             Kamet0
3           Squeezie
4            Domingo
            ...     
2495           T0XIA
2496         Datalgo
2497      JiloOziris
2498    DeejayMakina
2499    MisterDwight
Name: 0, Length: 2500, dtype: object

## Get the access token from twitch api

In [26]:
# Get the login from file in git ignore
client_id = twitch_api_login.Client_ID
client_secret = twitch_api_login.Secret

# get the access token
auth = authenticator.Auth(client_id, client_secret)
access_token = auth.get_auth()
access_token

'd2eutmez588jvung496mpvwhyaq8ko'

## Get call_api_service Class

In [27]:
call_api = ca.Call_api_service(client_id, access_token)

## Get the twitch_Id for Each Streamer

In [28]:
twitch_id = {}
for streamer in top_2500[0]:
    streamer_id = call_api.get_user_id(streamer)
    twitch_id[streamer] = streamer_id

In [29]:
#transform list to dataframe
df = pd.DataFrame.from_dict({'streamer': twitch_id.keys(), 'twitch_id': twitch_id.values()})

# saving the dataframe
df.to_csv('../streamers.csv', header=True, index=False)

## CONNECT TO BDD

In [32]:
import mysql.connector


bdd = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="twitch_user_map"
)

mycursor = bdd.cursor()

## INSERT DATA ON BDD

In [33]:
for index, row in df.iterrows():
    pseudo = row['streamer']
    id_twitch = row['twitch_id']
    val = (pseudo, id_twitch)
    sql = "INSERT INTO streamers (pseudo, id_twitch) VALUES (%s, %s)"

    mycursor.execute(sql, val)

    bdd.commit()